# 清理MR的Log檔

先讀取原本的log檔，把要的資料寫入到新的txt檔

In [1]:
import os
import requests
import pandas as pd
import time
import os
from io import StringIO
import warnings
warnings.filterwarnings("ignore")

file_path = 'mr_log'
file_log = 'MR'
file_log_new = 'MR_new'

exclude_keyword=[
    'Newtonsoft.Json', 
    'String ',
    'Rev:{"code',
    'Send:https',
    'System.N',
    '於',
    '券商回覆',
    '下單數量',
    '未定義',
    '內部例外',
    '多空比例',
    '獨立口數',
    '非同步下單',
    '同步下單',
    'Login Error',
    'ServerExit'
]

include_keyword = ['B 成交價格','S 成交價格']

f = open(file_path + f'/{file_log}.log',"r") 
lines = f.readlines() 
f.close()

f = open(file_path + f'/{file_log_new}.log',"w") 
for line in lines:
    match_exclude_keyword = False
    for word in exclude_keyword:
            if word in line:
                match_exclude_keyword = True
    
    match_include_keyword = False
    for word in include_keyword:
        if word in line:
            match_include_keyword = True
                
    if match_exclude_keyword == False and match_include_keyword == True:
        line = line.replace('[DEBUG] ',',')
        line = line.replace('[\x08.\x02] ',' ')
        line = line.replace(' 成交價格：',' , ')
        line = line.replace(' 成交數量：',' , ')
        line = line.replace(' , ','***')
        line = line.replace('[','')
        line = line.replace(']','')
        f.write(line) 
f.close()
print('整理完畢')

整理完畢


# 讀取新的txt檔，整理成dataframe格式並儲存

In [2]:
df_name = 'df_mr'
with open(file_path + f'/{file_log_new}.log', 'r') as f:
    txt=f.read()

# print(txt)

io_txt_info = StringIO(txt)

columns = ["時間","買賣","成交價","口數"]
new_df = pd.DataFrame(columns = columns)

for line in io_txt_info:
    line = line.replace('\n','')
    line = line.split('***')  # 清理資料完成
    
    df = pd.DataFrame([line], columns = columns) #每一行建立df
    new_df = new_df.append(df, ignore_index = True)
    
new_df.to_csv(file_path + f'/{df_name}.csv')
# print(new_df)

new_df

,時間,買賣,成交價,口數
0,"2021-11-04 02:45:34,164",B,17155,1
1,"2021-11-04 04:54:03,714",S,17223,1
2,"2021-11-04 09:01:08,422",S,17214,1
3,"2021-11-04 09:03:05,635",S,17209,1
4,"2021-11-04 09:50:36,884",S,17178,1
...,...,...,...,...
1952,"2022-03-05 00:00:02,548",S,17574,1
1953,"2022-03-05 01:10:14,620",B,17599,1
1954,"2022-03-05 04:30:02,866",S,17522,1
1955,"2022-03-05 04:53:01,580",B,17533,1


# 篩選資料

設定條件篩選出需要的資料，像是某段時間或是買單還是賣單

In [3]:
start_time = '2022-03-04 08:45'
end_time   = '2022-03-05 00:00' 
s_time_fliter = (new_df['時間'] > start_time)
e_time_fliter = (new_df['時間'] < end_time)

B_filter = (new_df['買賣'] == 'B')
S_filter = (new_df['買賣'] == 'S')

#多條件:
print(new_df[B_filter][s_time_fliter])
print(new_df[S_filter][s_time_fliter][e_time_fliter])

# 在範圍內的寫法 1，用 & 連接:
print(new_df[s_time_fliter & e_time_fliter])

# 在範圍內的寫法 2，用between的方式:
tb = new_df['時間'].between(start_time, end_time)
print(new_df[tb])

                           時間 買賣    成交價 口數
1944  2022-03-04 09:35:57,362  B  17732  1
1945  2022-03-04 09:50:27,712  B  17749  1
1946  2022-03-04 09:50:43,266  B  17758  1
1951  2022-03-04 21:30:13,152  B  17654  1
1953  2022-03-05 01:10:14,620  B  17599  1
1955  2022-03-05 04:53:01,580  B  17533  1
1956  2022-03-05 04:54:01,398  B  17548  1
                           時間 買賣    成交價 口數
1947  2022-03-04 11:33:12,576  S  17724  2
1948  2022-03-04 13:35:00,671  S  17693  1
1949  2022-03-04 13:35:01,092  S  17694  1
1950  2022-03-04 13:39:00,684  S  17688  1
                           時間 買賣    成交價 口數
1944  2022-03-04 09:35:57,362  B  17732  1
1945  2022-03-04 09:50:27,712  B  17749  1
1946  2022-03-04 09:50:43,266  B  17758  1
1947  2022-03-04 11:33:12,576  S  17724  2
1948  2022-03-04 13:35:00,671  S  17693  1
1949  2022-03-04 13:35:01,092  S  17694  1
1950  2022-03-04 13:39:00,684  S  17688  1
1951  2022-03-04 21:30:13,152  B  17654  1
                           時間 買賣    成交價 口數
1944  2022-